# Modèles de Recommandations.

## 1. Librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark import StorageLevel

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

## 2. Création d'une session Spark

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyALS") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.sql.shuffle.partitions", "8") \
    .master("local[*]") \
    .getOrCreate()

## 3. Chargement des données

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = spark.read.parquet("/content/drive/MyDrive/ml-32m/df.parquet")

In [8]:
df.show(5)

+-------+--------------------+--------------------+------+------+----------+--------------------+----+--------------------+
|movieId|               title|              genres|userId|rating| timestamp|         genres_list|year|    title_sans_annee|
+-------+--------------------+--------------------+------+------+----------+--------------------+----+--------------------+
|   8970|Finding Neverland...|               Drama| 62769|   5.0|1426961515|             [Drama]|2004|   Finding Neverland|
|  27815|Chorus, The (Chor...|               Drama| 62769|   3.5|1426959562|             [Drama]|2004|Chorus, The (Chor...|
|  30707|Million Dollar Ba...|               Drama| 62769|   4.0|1426959541|             [Drama]|2004| Million Dollar Baby|
|  33166|        Crash (2004)|         Crime|Drama| 62769|   5.0|1426961486|      [Crime, Drama]|2004|               Crash|
|  40819|Walk the Line (2005)|Drama|Musical|Rom...| 62769|   3.0|1426961490|[Drama, Musical, ...|2005|       Walk the Line|
+-------

## 4. Modélisation avec Spark MLlib : ALS

### 4.1. Préparation des données

In [9]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import DataFrame
from pyspark.sql.functions import rand

# Sous-échantillonage

df_sample = df.sample(withReplacement=False, fraction=0.5, seed=333)

Split par proportion stratifiée:
pour chaque utilisateur, on fait un split aléatoire ou temporel de type 80/20.
On s’assure que pour chaque utilisateur, il reste suffisamment d’interactions dans les deux ensembles.

Pourquoi? Parce qu'un split classique ne garantit pas la présence dans le test, d'interactions user-film utilisée pendant l'entraînement.


In [10]:
from pyspark.sql import Window
import pyspark.sql.functions as F

# Proportion du train
train_ratio = 0.8

# Création d'un rang temporel par utilisateur
window = Window.partitionBy("userId").orderBy(F.col("timestamp"))
df_strat = df_sample.withColumn("rank", F.row_number().over(window))

# Nombre d'interactions par utilisateur
user_counts = df_strat.groupBy("userId").agg(F.max("rank").alias("count_per_user_join"))

# Jointure avec df_strat pour chaque ligne
df_strat = df_strat.join(user_counts, on="userId", how="left")

# Seuil train/test par utilisateur
df_strat = df_strat.withColumn(
    "train_cutoff",
    (F.col("count_per_user_join") * train_ratio).cast("int")
)

# Split stratifié : les ranks <= cutoff vont au train, le reste au test
train = df_strat.filter(F.col("rank") <= F.col("train_cutoff"))
test  = df_strat.filter(F.col("rank") >  F.col("train_cutoff"))

In [11]:
# Chemins de sauvegarde dans ton Google Drive
train_path = "/content/drive/MyDrive/ml-32m/train_data"
test_path = "/content/drive/MyDrive/ml-32m/test_data"

# Sauvegarde au format Parquet (format efficace et recommandé pour Spark)
train.write.mode("overwrite").parquet(train_path)
test.write.mode("overwrite").parquet(test_path)

liste des utilisateurs dans le test

In [ ]:
users_test = test.select("userId").distinct()

liste des utilisateurs dans le train

In [ ]:
users_train = train.select("userId").distinct()

Recherche d'utilisateurs du test absents du train

In [ ]:
users_missing = users_test.join(users_train, on="userId", how="left_anti")
users_missing.show()  # affichge des userId à problème

+------+
|userId|
+------+
+------+



### 4.2. Entraînement de l'ALS (70% des données)

In [ ]:
# Extraction des colonnes nécessaires

als_train = train.select("userId", "movieId", "rating")
als_test  = test.select("userId", "movieId", "rating")


In [ ]:
# ALS (grid search sur les hyper paramètres)

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Initialisation du modèle ALS
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop",
    nonnegative=True
)

# Recherche sur rank, regParam, maxIter
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 20]) \
    .addGrid(als.regParam, [0.01, 0.05, 0.1]) \
    .addGrid(als.maxIter, [5, 10]) \
    .build()

# RMSE comme critère de sélection
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

# Cross-validation
cv = CrossValidator(
    estimator=als,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=2
)

# Entraînement pour obtenir les meilleurs hyperparamètres
cv_model = cv.fit(als_train)

# Meilleur modèle
best_als_model = cv_model.bestModel

In [ ]:
# Sauvegarde du meilleur modèle dans un fichier

best_als_model.write().overwrite().save("/content/drive/MyDrive/ml-32m/best_als_model")

In [ ]:
# Chargement du meilleur modèle

from pyspark.ml.recommendation import ALSModel

best_als_model = ALSModel.load("/content/drive/MyDrive/ml-32m/best_als_model")


In [ ]:
# Récupérer la valeur de chaque hyperparamètre
best_rank     = best_als_model._java_obj.parent().getRank()
best_regparam = best_als_model._java_obj.parent().getRegParam()
best_maxiter  = best_als_model._java_obj.parent().getMaxIter()

print(f"Best rank: {best_rank}")
print(f"Best regParam: {best_regparam}")
print(f"Best maxIter: {best_maxiter}")

Best rank: 20
Best regParam: 0.1
Best maxIter: 10


### 4.3. Prédictions sur le test

In [ ]:
# Prédiction sur le test set
pred = best_als_model.transform(test)

# Calcul du RMSE
rmse_best = evaluator.evaluate(pred)
print(f"RMSE du meilleur modèle sur le test: {rmse_best:.4f}")

RMSE du meilleur modèle sur le test: 0.8331


In [ ]:
# RMSE moyens de chaque configuration de la grille (sur les folds CV)

print("RMSE moyens sur la grille (validation croisée):", cv_model.avgMetrics)

RMSE moyens sur la grille (validation croisée): [np.float64(0.8728500501005159), np.float64(0.8495061871870658), np.float64(0.8586408014429849), np.float64(0.8365147767124903), np.float64(0.8468710866059251), np.float64(0.8328230406778426), np.float64(0.8748739987335976), np.float64(0.8664983749685551), np.float64(0.8603939721664892), np.float64(0.8340971326443893), np.float64(0.8485555402573826), np.float64(0.8262200448722843), np.float64(0.8925770618732198), np.float64(0.8991618234813956), np.float64(0.8615430240322999), np.float64(0.8345654896407388), np.float64(0.8536234982073747), np.float64(0.824447213078618)]


### 4.4. Recommandations

In [ ]:
# Top 5 des utilisateurs susceptibles d'apprécier chaque film
recommandations_items = best_als_model.recommendForAllItems(5)
recommandations_items.show(truncate=False)

+-------+------------------------------------------------------------------------------------------------------+
|movieId|recommendations                                                                                       |
+-------+------------------------------------------------------------------------------------------------------+
|1      |[{22571, 5.710531}, {48515, 5.449351}, {36189, 5.4239635}, {197988, 5.418619}, {17782, 5.410211}]     |
|3      |[{60356, 4.94844}, {14, 4.9207144}, {155152, 4.8754106}, {126492, 4.830433}, {128408, 4.8092833}]     |
|5      |[{126492, 4.8918767}, {49191, 4.853008}, {144110, 4.845821}, {135288, 4.8327594}, {99897, 4.782475}]  |
|6      |[{66795, 5.316153}, {139881, 5.2190113}, {184156, 5.1247377}, {160024, 5.0900216}, {107562, 5.087736}]|
|7      |[{91084, 4.9351034}, {36189, 4.907238}, {149035, 4.8563}, {144110, 4.8562903}, {135288, 4.85153}]     |
|9      |[{128408, 4.9791055}, {131535, 4.6913066}, {155152, 4.65787}, {188652, 4.6452737}, {218

In [ ]:
# Top 5 des recommandations par utilisateur
recommandations = best_als_model.recommendForAllUsers(5)
recommandations.show(truncate=False)

+------+---------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                          |
+------+---------------------------------------------------------------------------------------------------------+
|5     |[{193389, 4.763201}, {154280, 4.7091417}, {216753, 4.7051306}, {140353, 4.649042}, {222368, 4.5872188}]  |
|6     |[{154280, 7.5390973}, {222368, 7.5318093}, {205277, 7.5318093}, {265656, 7.3779125}, {208459, 7.0270233}]|
|9     |[{196167, 5.9601665}, {86288, 5.763021}, {190707, 5.7019267}, {265364, 5.6343427}, {138224, 5.5507965}]  |
|10    |[{151989, 4.5009556}, {86288, 4.4064307}, {196167, 4.3716984}, {265364, 4.3121934}, {192261, 4.2617397}] |
|12    |[{282453, 4.2550135}, {210621, 4.2102065}, {192949, 4.1146603}, {159896, 4.1146603}, {115987, 4.1146603}]|
|13    |[{86288, 5.4072747}, {274047, 5.3746395}, {196167, 5.1407795}, {182527, 

### 4.5. Evaluations des recommandations

Les recommandations fournies par l'ALS incluent probablement des films déjà vus par l'utilisateur, parce que l'algorithme ALS de Spark ne filtre pas par défaut les anciens items pour chaque utilisateur lors du calcul des recommandations.

Pour avoir des métriques d'évaluation qui montre la réelle perfromance de notre modèle, on décide de filtrer les résultats de l'ALS pour ne garder que les films non précédemment vus par chaque utilisateur.


In [ ]:
# Génération de k = 5 recommandations natives de l'ALS

k = 5
user_recs = best_als_model.recommendForAllUsers(k)

In [ ]:
df = spark.read.parquet("/content/drive/MyDrive/ml-32m/df.parquet")

In [ ]:
from pyspark.sql.functions import arrays_zip, explode, col

# Accéder aux movieId et rating depuis les recommandations
user_recs_exploded = (
    user_recs
    .select('userId', explode('recommendations').alias('rec'))
    .select('userId', col('rec.movieId').alias('movieId'), col('rec.rating').alias('rating'))
)
already_watched = train.select('userId', 'movieId')
final_recs = user_recs_exploded.join(already_watched, ['userId', 'movieId'], 'left_anti')

In [ ]:
# Filtrage des films déjà vus

vue = train.withColumn("vu", col("rating"))
recs_nouveaux = user_recs_exploded.join(vue.select("userId", "movieId"), ["userId", "movieId"], how="left_anti")

Reconstitution d'un top-k sans doublon

In [ ]:
# Remplacement des predictions par les ratings
w = Window.partitionBy("userId").orderBy(F.desc("rating"))
final_recs = recs_nouveaux.withColumn("rank", F.row_number().over(w)).filter(F.col("rank") <= k)
final_recs.show(5)

+------+-------+---------+----+
|userId|movieId|   rating|rank|
+------+-------+---------+----+
|     1| 164937| 5.283037|   1|
|     1| 193918| 5.109409|   2|
|     1| 155641|5.0890317|   3|
|     1| 184653|5.0705643|   4|
|     1| 168552| 4.846283|   5|
+------+-------+---------+----+
only showing top 5 rows



On obtiens ainsi le top-k recommandations neuves (hors films déjà vus) pour chaque utilisateur.

Métriques d'évaluation: top k, recall@k, precision, F1, coverage

In [ ]:
from pyspark.sql import functions as F

K = 10  # nombre de recommandations par utilisateur

In [ ]:
# Regroupement des recommandations par utilisateur

recs_k = final_recs.groupBy("userId").agg(F.collect_list("movieId").alias("rec_movies"))

In [ ]:
# Création de test_positive (ensemble des films appréciés dans le test par utilisateur)

from pyspark.sql import functions as F

seuil_like = 3  # seuil pour 'aimer' un film
test_positive = (
    test.filter(F.col("rating") >= seuil_like)
        .groupBy("userId")
        .agg(F.collect_set("movieId").alias("test_movies"))
)

In [ ]:
# Jointure avec la vérité de test

eval_df = recs_k.join(test_positive, "userId", "inner")

In [ ]:
def precision_recall_f1(rec_movies, test_movies):
    rec_set = set(rec_movies or [])
    test_set = set(test_movies or [])
    nb_hits = len(rec_set & test_set)
    precision = nb_hits / len(rec_set) if rec_set else 0
    recall = nb_hits / len(test_set) if test_set else 0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
    return float(precision), float(recall), float(f1), nb_hits

from pyspark.sql.types import StructType, StructField, FloatType, IntegerType
from pyspark.sql.functions import udf

schema = StructType([
    StructField("precision", FloatType(), False),
    StructField("recall", FloatType(), False),
    StructField("f1", FloatType(), False),
    StructField("nb_hits", IntegerType(), False)
])

prf_udf = udf(precision_recall_f1, schema)

eval_metrics = eval_df.withColumn(
    "metrics", prf_udf("rec_movies", "test_movies")
).select(
    "userId",
    "metrics.precision",
    "metrics.recall",
    "metrics.f1",
    "metrics.nb_hits"
)


In [ ]:
K = 10

metrics_rdd = eval_df.rdd.map(lambda row: precision_recall_f1(
    row["rec_movies"][:K],  # coupe à K
    row["test_movies"]
))

precisions = metrics_rdd.map(lambda x: x[0]).collect()
recalls    = metrics_rdd.map(lambda x: x[1]).collect()
f1s        = metrics_rdd.map(lambda x: x[2]).collect()

precision_at_k = sum(precisions) / len(precisions)
recall_at_k    = sum(recalls) / len(recalls)
f1_at_k        = sum(f1s) / len(f1s)

print(f"Precision@{K}: {precision_at_k}")
print(f"Recall@{K}: {recall_at_k}")
print(f"F1@{K}: {f1_at_k}")

Precision@10: 1.2991230919129589e-05
Recall@10: 2.2196135526941244e-06
F1@10: 2.602483360853983e-06


In [ ]:
results = eval_metrics.agg(
    F.mean("precision").alias(f'Precision@{K}'),
    F.mean("recall").alias(f'Recall@{K}'),
    F.mean("f1").alias(f'F1@{K}')
)
results.show()


+--------------------+--------------------+--------------------+
|        Precision@10|           Recall@10|               F1@10|
+--------------------+--------------------+--------------------+
|1.299123118716956...|2.219613567998886E-6|2.602483378217199E-6|
+--------------------+--------------------+--------------------+



In [ ]:
# Liste unique de tous les films recommandés
all_recommended = final_recs.select("movieId").distinct()
# Nombre de films dans le catalogue à recommander (ex : tous les films testés, ou tous)
total_catalog = df_sample.select("movieId").distinct().count()

coverage = all_recommended.count() / total_catalog
print(f"Coverage@{K} : {coverage:.4f}")

Coverage@10 : 0.0119


In [ ]:
# Regroupement du top@k

from pyspark.sql import functions as F, Window

K = 10  # top@k à calculer

# Si ce n'est pas déjà fait, on trie et on rajoute un rang pour les k premiers films recommandés par utilisateur
w = Window.partitionBy("userId").orderBy(F.desc("rating"))
topk_recs = final_recs.withColumn("rank", F.row_number().over(w)).filter(F.col("rank") <= K)

# Regroupe les recommandations du top@k par utilisateur
recs_k = topk_recs.groupBy("userId").agg(F.collect_set("movieId").alias("rec_movies"))

In [ ]:
# Jointure avec la vérité test

eval_df = recs_k.join(test_positive, "userId", "inner")

In [ ]:
# Indice de Dice pour chaque utilisateur

from pyspark.sql.types import FloatType

def dice_index(rec_movies, test_movies):
    set_rec = set(rec_movies or [])
    set_test = set(test_movies or [])
    inter = set_rec & set_test
    denom = len(set_rec) + len(set_test)
    return float(2 * len(inter) / denom) if denom > 0 else 0.0

from pyspark.sql.functions import udf
dice_udf = udf(dice_index, FloatType())

eval_df = eval_df.withColumn("dice", dice_udf("rec_movies", "test_movies"))

In [ ]:
# Résultat
eval_df.select("userId", "rec_movies", "test_movies", "dice").show(10, truncate=False)

# Pour la moyenne globale sur tous les utilisateurs, si souhaité :
eval_df.agg(F.mean("dice").alias("mean_dice")).show()


+------+----------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|userId|rec_movies                              |test_movies                                                                                                                                                                                                                                                                                                                                                                                           

## 5. Recommandation de films basée sur le contenu (genres, TF-IDF, similarité cosinus)

In [13]:
# Extraction et vectorisation des genres (prétraitement)

from pyspark.sql.functions import split, col
train = train.withColumn("genres_list", split(col("genres"), "\|"))
test  = test.withColumn("genres_list", split(col("genres"), "\|"))

In [14]:
# Vectorisation TF-IDF: application du même modèle TF-IDF sur tout le catalogue de films à recommander et sur le train

from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline

cv = CountVectorizer(inputCol="genres_list", outputCol="raw_features", minDF=2)  # minDF réduit le bruit
idf = IDF(inputCol="raw_features", outputCol="features")
pipeline = Pipeline(stages=[cv, idf])

tfidf_model = pipeline.fit(train)
train_tfidf = tfidf_model.transform(train)
test_tfidf = tfidf_model.transform(test)

In [15]:
#Construction des Profils Utilisateur

    #Filtrage des films appréciés dans le train :

from pyspark.sql.functions import col

seuil_like = 4.0
liked_train = train_tfidf.filter(col("rating") >= seuil_like)

In [16]:
# Agrégation (profil moyen) :
# Spark n’offre pas de moyenne native sur un vector. On les convertit en Array pour faire une moyenne proprement.


from pyspark.sql.functions import collect_list, udf
import numpy as np
from pyspark.sql.types import ArrayType, DoubleType

def mean_vectors(arrs):
    if not arrs: return []
    arrs = [np.array(a.toArray()) if hasattr(a, 'toArray') else np.array(a) for a in arrs]
    return (np.mean(arrs, axis=0)).tolist()

mean_vectors_udf = udf(mean_vectors, ArrayType(DoubleType()))

user_profiles = liked_train.groupBy("userId").agg(
    collect_list("features").alias("arrs")
).withColumn("user_profile", mean_vectors_udf("arrs"))

In [30]:
# Catalogue de recommandation (films candidats), en utilisant tous les films du catalogue, pas seulement le train

movies = spark.read.csv("/content/drive/MyDrive/ml-32m/movies.csv", header=True, inferSchema=True)
df_catalog = movies.select("movieId", "genres").distinct()
df_catalog = df_catalog.withColumn("genres_list", split(col("genres"), "\|"))
catalog_tfidf = tfidf_model.transform(df_catalog).select("movieId", "features")

In [31]:
# Calcul de la similarité cosinus

    # Implémentation UDF

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import numpy as np

def cosine_sim(v1, v2):
    arr1, arr2 = np.array(v1), np.array(v2)
    num = np.dot(arr1, arr2)
    denom = np.linalg.norm(arr1) * np.linalg.norm(arr2)
    return float(num / denom) if denom else 0.0

cosine_udf = udf(cosine_sim, DoubleType())

In [32]:
# Croisement profils utilisateurs et catalogue :


user_films = user_profiles.crossJoin(catalog_tfidf)
user_films = user_films.withColumn(
    "cosine_sim",
    cosine_udf("user_profile", "features")
)

In [33]:
#Filtrage des films déjà vus

    # Ejection des films déjà notés par chaque utilisateur :

df_seen = train.select("userId", "movieId").distinct()
user_films = user_films.join(df_seen, on=["userId", "movieId"], how="left_anti")

In [34]:
# Top-N recommandations par similarité

    # Classement et coupe pour Top-N (exemple : N=10) :


from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

N = 10
w = Window.partitionBy("userId").orderBy(col("cosine_sim").desc())
topn_recs = user_films.withColumn("rank", row_number().over(w)) \
    .filter(col("rank") <= N) \
    .select("userId", "movieId", "cosine_sim", "rank")

In [35]:
# Vérité de test (films appréciés dans le test)

from pyspark.sql import functions as F
test_positive = test.filter(col("rating") >= seuil_like).groupBy("userId") \
    .agg(F.collect_set("movieId").alias("test_movies"))

In [36]:
# Jointure recommandations et vérité

eval_df = topn_recs.join(test_positive, "userId", "left") \
    .withColumn("hit", F.expr("array_contains(test_movies, movieId)").cast("int"))

In [37]:
# Recommandations de k films similaires à un film donné
'''Pour un film cible, il suffit de :

    Extraire le vecteur TF-IDF (basé sur les genres) correspondant à ce film dans le catalogue.

    Calculer la similarité cosinus entre ce vecteur et celui de tous les autres films du catalogue (hors lui-même).

    Trier les scores de similarité décroissants.

    Afficher les k premiers résultats.'''

from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
import numpy as np

movie_id_cible = 30707  # ID du film pour lequel on veut des recommandations
k = 5

# Récupération du profil TF-IDF du film cible
film_query_vec = catalog_tfidf.filter(col("movieId") == movie_id_cible).select("features").collect()[0][0]

# Definition de lan fonction cosine_sim UDF qui capture le film_query_vec
def cosine_sim_udf_factory(query_vec):
    def cosine_sim(v1):
        arr1, arr2 = np.array(v1.toArray()), np.array(query_vec.toArray())
        num = np.dot(arr1, arr2)
        denom = np.linalg.norm(arr1) * np.linalg.norm(arr2)
        return float(num / denom) if denom else 0.0
    return udf(cosine_sim, DoubleType())

# Création de l'instance UDF avec le vecteur de requête spécifique
cosine_udf_instance = cosine_sim_udf_factory(film_query_vec)

# Calcul de la similarité cosinus en utilisant l'instance UDF
recs_film = catalog_tfidf.withColumn(
    "cosine_sim",
    cosine_udf_instance(col("features")) # Ne passer que la colonne desfeatures
)


# Suppression du film d'origine et sélection des k plus proches
recs_film = recs_film.filter(col("movieId") != movie_id_cible) \
    .orderBy(col("cosine_sim").desc()) \
    .limit(k)

recs_film.select("movieId", "cosine_sim").show()

+-------+----------+
|movieId|cosine_sim|
+-------+----------+
|    396|       1.0|
|    636|       1.0|
|    452|       1.0|
|    388|       1.0|
|    491|       1.0|
+-------+----------+



In [41]:
# Affichage des films proches de movieId = 30707 (Million Dollar Baby)

movies_unique = movies.select("movieId", "title", "genres").dropDuplicates(["movieId"])

recs_film_with_infos = recs_film.join(movies_unique, on="movieId", how="left")

recs_film_with_infos.select("movieId", "title", "genres", "cosine_sim").show(truncate=False)


+-------+------------------------------+------+----------+
|movieId|title                         |genres|cosine_sim|
+-------+------------------------------+------+----------+
|396    |Fall Time (1995)              |Drama |1.0       |
|636    |Frisk (1995)                  |Drama |1.0       |
|452    |Widows' Peak (1994)           |Drama |1.0       |
|388    |Boys Life (1995)              |Drama |1.0       |
|491    |Man Without a Face, The (1993)|Drama |1.0       |
+-------+------------------------------+------+----------+



In [42]:
from pyspark.sql.functions import split

recs_film_with_infos = recs_film_with_infos.withColumn("genres_list", split(col("genres"), "\|"))

In [43]:
recs_film_with_infos.select("movieId", "title", "genres_list", "cosine_sim").show(truncate=False)

+-------+------------------------------+-----------+----------+
|movieId|title                         |genres_list|cosine_sim|
+-------+------------------------------+-----------+----------+
|396    |Fall Time (1995)              |[Drama]    |1.0       |
|636    |Frisk (1995)                  |[Drama]    |1.0       |
|452    |Widows' Peak (1994)           |[Drama]    |1.0       |
|388    |Boys Life (1995)              |[Drama]    |1.0       |
|491    |Man Without a Face, The (1993)|[Drama]    |1.0       |
+-------+------------------------------+-----------+----------+



Evaluation

In [44]:
ratings = spark.read.csv("/content/drive/MyDrive/ml-32m/ratings.csv", header=True, inferSchema=True)

In [45]:
from pyspark.sql.functions import rand

# On trie et sépare les notes utilisateur aléatoirement
ratings = ratings.orderBy(rand())
training = ratings.sampleBy("userId", fractions={u: 0.8 for u in ratings.select("userId").distinct().rdd.flatMap(lambda x: x).collect()}, seed=42)
test = ratings.subtract(training)

In [46]:
# Fonction de recommandation par utilisateur


def get_top_k_recommendations(user_id, k):
    # 1. Récupère les films notés par l'utilisateur dans le train set
    user_history = training.filter(col("userId") == user_id).select("movieId").collect()
    watched_ids = [row["movieId"] for row in user_history]

    # 2. Récupère les vecteurs TF-IDF des films vus
    user_profile = catalog_tfidf.filter(col("movieId").isin(watched_ids))

    # 3. Calcule une moyenne du profil utilisateur
    from pyspark.ml.linalg import Vectors, DenseVector
    import numpy as np

    vectors = np.array([vec["features"].toArray() for vec in user_profile.collect()])
    if len(vectors) == 0:
        return []
    avg_profile = Vectors.dense(np.mean(vectors, axis=0))

    # 4. Calcule la similarité avec tout le catalogue
    cosine_udf = cosine_sim_udf_factory(avg_profile)
    scores = catalog_tfidf.withColumn("cosine_sim", cosine_udf(col("features")))

    # 5. Retire les films déjà vus
    scores = scores.filter(~col("movieId").isin(watched_ids))

    # 6. Retourne les k premiers movieId
    top_k = scores.orderBy(col("cosine_sim").desc()).limit(k).select("movieId").collect()
    return [row["movieId"] for row in top_k]


Precision@k et Recall@k

Coverage

Indice de Dice

In [ ]:
# Synthèse des métriques

print(f"Precision@{k} = {precision_at_k:.4f}")
print(f"Recall@{k} = {recall_at_k:.4f}")
print(f"Coverage = {coverage:.4f}")
print(f"Dice coefficient = {dice_coeff:.4f}")

## 6. Recommandation Basée sur les Proximités Utilisateurs (User-KNN)

In [ ]:
# Préparation des données

from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer
import pandas as pd
import numpy as np

# Set a higher value for spark.sql.pivotMaxValues
spark.conf.set("spark.sql.pivotMaxValues", 100000) # Set to a value higher than the number of unique movies

# Extraire du DataFrame PySpark (pour manipuler numpy)
pivot_df = train.groupBy("userId").pivot("movieId").agg(F.first("rating"))
pivot_pd = pivot_df.toPandas().set_index("userId").fillna(0)
user_item_matrix = pivot_pd.values  # Matrice numpy users x items
user_ids = pivot_pd.index.values

In [ ]:
# Mesure de similarité entre utilisateurs (similarité cosinus)

from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(user_item_matrix)
# Diagonale = 1 (identité utilisateur)

In [ ]:
# Recherche des k voisins les plus proches

k = 10  # nombre de voisins à considérer
top_k_indices = np.argsort(-similarity_matrix, axis=1)[:, 1:k+1]  # évite l'utilisateur lui-même (colonne 0)

In [ ]:
'''Génération des recommandations

Pour un utilisateur donné :

    On aboutit à :

        Ses k voisins les plus similaires.

        Les notes qu'ils ont donné à chaque film.

        On moyenne/pondère les notes pour chaque film non vu par l'utilisateur cible.

        On trie les scores et propose les meilleurs films.
'''

def recommend_for_user(user_idx, user_item_matrix, top_k_indices, user_ids, N=10):
    neighbors = top_k_indices[user_idx]
    # Moyenne (ou pondérée par similarité) des notes des voisins, sur les films non vus
    neighbor_ratings = user_item_matrix[neighbors]
    user_ratings = user_item_matrix[user_idx]
    already_seen = set(np.where(user_ratings > 0)[0])

    # Calcul du score moyen
    mean_scores = neighbor_ratings.mean(axis=0)
    scores = [(i, score) for i, score in enumerate(mean_scores) if i not in already_seen]
    # Top-N recommandations (par score décroissant)
    top_n = sorted(scores, key=lambda x: -x[1])[:N]
    return top_n  # indices des films à recommander

# Pour tous les utilisateurs
recs_by_user = {user_ids[i]: recommend_for_user(i, user_item_matrix, top_k_indices, user_ids) for i in range(len(user_ids))}

Recommandation Hybride : KNN Utilisateur + Genres

In [ ]:
# Préparer les données utilisateur-film et d'intégration des genres

from pyspark.sql.functions import split, col, collect_set

# train : userId, movieId, rating, genres (séparés par |)
train = train.withColumn("genres_list", split(col("genres"), "\\|"))

NameError: name 'train' is not defined

In [ ]:
# Établir le profil « genre préféré » de chaque utilisateur

user_genres = train.filter(col("rating") >= 4) \
    .select("userId", "genres_list") \
    .withColumn("genre", F.explode("genres_list")) \
    .groupBy("userId") \
    .agg(collect_set("genre").alias("preferred_genres"))

In [ ]:
# Construire la matrice user-item via pivot (ratings explicites uniquement)

user_item_df = train.groupBy("userId").pivot("movieId").agg(F.first("rating"))
user_item_pd = user_item_df.toPandas().set_index("userId").fillna(0)
user_ids = user_item_pd.index.values

In [ ]:
# Calculer la similarité entre utilisateurs (KNN collaboratif seulement)

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

user_item_matrix = user_item_pd.values
sim_matrix = cosine_similarity(user_item_matrix)
# On force la diagonale à 0 (évite d'être son propre voisin)
np.fill_diagonal(sim_matrix, 0)
k = 10  # voisins

# Indices des k plus proches voisins pour chaque utilisateur
topk_neighbor_indices = np.argsort(-sim_matrix, axis=1)[:, :k]

In [ ]:
# Gérer le cold start et la diversité dans la recommandation

# - Si un utilisateur n'a PAS de voisins (cold start), CF: => recommandation popularité et/ou par genre préféré
# - Pour la diversité, on filtre ou pondère la liste finale selon qu'un film apporte un genre « différent » des habitudes

def get_user_genres(user_id):
    # mapping userId -> genres préférés (set)
    row = user_genres.filter(col("userId") == user_id).collect()
    return set(row[0]["preferred_genres"]) if row else set()

movies_genres = train.select("movieId", "genres_list").distinct()
movies_genres_dict = dict(movies_genres.collect())

In [ ]:
# Générer les recommandations hybrides pour un utilisateur


def recommend_knn_genres(user_idx, N=10):
    neighbors = topk_neighbor_indices[user_idx]
    user_seen = set(np.where(user_item_matrix[user_idx] > 0)[0])
    scores = np.zeros(user_item_matrix.shape[1])

    for n_idx in neighbors:
        neighbor_ratings = user_item_matrix[n_idx]
        scores += neighbor_ratings

    # On ne recommande pas les films déjà vus
    for idx in user_seen:
        scores[idx] = -np.inf

    # On priorise : genre jamais vu pour l'utilisateur (diversité)
    user_id = user_ids[user_idx]
    user_fav_genres = get_user_genres(user_id)

    movie_indices = np.argsort(-scores)
    recs = []
    for i in movie_indices:
        if len(recs) >= N:
            break
        movie_id = user_item_pd.columns[i]
        film_genres = set(movies_genres_dict.get(movie_id, []))
        # Option diversity : push si au moins un genre nouveau pour user
        if len(film_genres - user_fav_genres) > 0 or not user_fav_genres:
            recs.append((movie_id, scores[i]))
    # Cold start si pas de recs : on propose les k films populaires alignés avec genres préférés
    if len(recs) == 0:
        for i in movie_indices:
            movie_id = user_item_pd.columns[i]
            film_genres = set(movies_genres_dict.get(movie_id, []))
            if len(film_genres & user_fav_genres) > 0:
                recs.append((movie_id, scores[i]))
                if len(recs) >= N:
                    break
    return recs[:N]

In [ ]:
# Appliquer à tous les utilisateurs

recommendations = {user_ids[i]: recommend_knn_genres(i, N=10) for i in range(len(user_ids))}